# Where to open a gym?

## Introduction

<p>Problem: A client is looking to open a gym in Ottawa and is looking for advice about where it should be located.</p>

<p>Ideally the location would be in an area that appears to be currently underserved by the availability of nearby gyms.
For this analysis, we will use the following information:</p>
<ul>
    <li>the number of gyms in each area in Ottawa using Forward Sortatation Areas (FSAs)</li>
    <li>the neighbourhood name(s) of each FSA (retrieved from Wikipedia)</li>
    <li>the population of each each FSA using the 2016 census</li>
    <li>the foursquare api to retrieve the current number of gyms per FSA</li>
</ul>
<p>Using the above information, we will cluster neighborhoods to find areas with a relatively high number of gyms and compare it to their population. We will use this data with the other areas to see if there is the potential to support another gym with their current population.</p>

# Data used in this project
<p>Wikipedia: <a href=https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_K> list of postal codes of Canada starting with K </a> to get neighbourhood names for each Ottawa FSA.</p>
<p><a href=https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/hlt-fst/pd-pl/comprehensive.cfm> Statistics Canada </a> to get the population data for each FSA</p>    
<p>geopy: to get lat/long coordinates for each FSA</p>
<p>FourSquare Api: to get the number of gyms per FSA </p>

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

## Scraping Wikipedia for the Postal Codes in Ottawa

Import the lxml file from wikipedia

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_K').text
soup = BeautifulSoup(source, 'lxml')


Test to find table and first postal code

In [3]:
table = soup.find('table')
postcode = table.td.text
print(postcode)

K1AGovernment of CanadaOttawa and Gatineau offices (partly in QC)



Create a list for the postal code, city, and neighbourhood. 
Add them to a pandas dataframe.

In [4]:
postalCode = []
city = []
neighbourhood = []


In [5]:
for row in soup.find('table').find_all('tr'):
    items = row.find_all('td')
    for i in items:
        contents = i.text
        if "Not assigned" in contents: continue #remove any postal codes not assigned
        if "Ottawa" not in contents: continue #remove any postal codes not in Ottawa
        if "partly in QC" in contents: continue #remove any postal code partially in Quebec
        postalCode.append(contents[:3]) #first three characters are the postal code
        city.append(contents[3:9]) #next 6 characters are 'Ottawa'
        neighbourhood.append(contents[contents.find("(")+1:contents.find(")")].replace(' /', ',')) #Neighbouthoods are found between parentheses, change all / to ,

In [6]:
ottawa_df = pd.DataFrame({"PostalCode": postalCode,
                          "City": city,
                          "Neighbourhood":neighbourhood})
ottawa_df.head(10)

,PostalCode,City,Neighbourhood
0,K2A,Ottawa,"Highland Park, McKellar Park,Westboro,Glabar P..."
1,K4A,Ottawa,Fallingbrook
2,K1B,Ottawa,"Blackburn Hamlet, Pine View, Sheffield Glen"
3,K2B,Ottawa,"Britannia,Whitehaven, Bayshore, Pinecrest"
4,K4B,Ottawa,Navan
5,K1C,Ottawa,Orleans
6,K2C,Ottawa,"Queensway, Copeland Park, Central Park, Bel Ai..."
7,K4C,Ottawa,Cumberland
8,K1E,Ottawa,Queenswood
9,K2E,Ottawa,Eastern Nepean: Fisher Heights/ Parkwood Hills...


In [7]:
for nh in ottawa_df['Neighbourhood']:
    print(nh)

Highland Park, McKellar Park,Westboro,Glabar Park,Carlingwood
Fallingbrook
Blackburn Hamlet, Pine View, Sheffield Glen
Britannia,Whitehaven, Bayshore, Pinecrest
Navan
Orleans
Queensway, Copeland Park, Central Park, Bel Air,Carleton Heights
Cumberland
Queenswood
Eastern Nepean: Fisher Heights/ Parkwood Hills, Borden Farm,Pine Glen 
Riverview, Hawthorne, Canterbury, Hunt Club Park
Centrepointe, Meadowlands, City View, Craig Henry, Tangelwood, Grenfell Glen, Davidson Heights
Alta Vista, Billings Bridge
Bells Corners, Arlington Woods/Redwood, Qualicum, Crystal Beach
Beacon Hill, Cyrville, Carson Grove
Barrhaven
Overbrook, Forbes, Manor Park, Viscount Alexander Park, Finter Quarries
Beaverbrook, South March
Vanier, McKay Lake area
Katimavik-Hazeldean, Glen Cairn
Rockcliffe Park, New Edinburgh
Bridlewood
Manotick
Lower Town, Byward Market, Sandy Hill, University of Ottawa
Downtown
Centretown
Greely
Dalhousie Ward
Fallowfield Village, Cedarhill Estates, Orchard Estates
The Glebe, Old Ottawa S

Print the number of rows in the dataframe

In [8]:
ottawa_df.shape

(40, 3)

# Import Population Data
<p> Accessed from <a href=https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/hlt-fst/pd-pl/comprehensive.cfm> Statistics Canada </a></p>

In [9]:
pop_df = pd.read_csv('https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/hlt-fst/pd-pl/Tables/CompFile.cfm?Lang=Eng&T=1201&OFT=FULLCSV', encoding='utf-8')

In [10]:
pop_df.head()

,Geographic code,Geographic name,Province or territory,"Incompletely enumerated Indian reserves and Indian settlements, 2016","Population, 2016","Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016"
0,01,Canada,NaN,T,35151728.0,15412443.0,14072079.0
1,A0A,A0A,Newfoundland and Labrador,NaN,46587.0,26155.0,19426.0
2,A0B,A0B,Newfoundland and Labrador,NaN,19792.0,13658.0,8792.0
3,A0C,A0C,Newfoundland and Labrador,NaN,12587.0,8010.0,5606.0
4,A0E,A0E,Newfoundland and Labrador,NaN,22294.0,12293.0,9603.0


In [11]:
pop_df.rename(columns = {"Geographic name":"PostalCode"}, inplace = True)
pop_df.head()

,Geographic code,PostalCode,Province or territory,"Incompletely enumerated Indian reserves and Indian settlements, 2016","Population, 2016","Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016"
0,01,Canada,NaN,T,35151728.0,15412443.0,14072079.0
1,A0A,A0A,Newfoundland and Labrador,NaN,46587.0,26155.0,19426.0
2,A0B,A0B,Newfoundland and Labrador,NaN,19792.0,13658.0,8792.0
3,A0C,A0C,Newfoundland and Labrador,NaN,12587.0,8010.0,5606.0
4,A0E,A0E,Newfoundland and Labrador,NaN,22294.0,12293.0,9603.0


In [12]:
unwcol = ['Geographic code','Province or territory','Incompletely enumerated Indian reserves and Indian settlements, 2016']
pop_df.drop(unwcol, inplace=True, axis=1)
pop_df.head()

,PostalCode,"Population, 2016","Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016"
0,Canada,35151728.0,15412443.0,14072079.0
1,A0A,46587.0,26155.0,19426.0
2,A0B,19792.0,13658.0,8792.0
3,A0C,12587.0,8010.0,5606.0
4,A0E,22294.0,12293.0,9603.0


In [13]:
ottawapop_df = ottawa_df.merge(pop_df, on="PostalCode", how="left")
ottawapop_df.head()

,PostalCode,City,Neighbourhood,"Population, 2016","Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016"
0,K2A,Ottawa,"Highland Park, McKellar Park,Westboro,Glabar P...",16790.0,7433.0,7205.0
1,K4A,Ottawa,Fallingbrook,54524.0,18677.0,18544.0
2,K1B,Ottawa,"Blackburn Hamlet, Pine View, Sheffield Glen",17110.0,6826.0,6611.0
3,K2B,Ottawa,"Britannia,Whitehaven, Bayshore, Pinecrest",32260.0,15640.0,14719.0
4,K4B,Ottawa,Navan,4793.0,1771.0,1732.0


# Import Ottawa Coordinates

In [14]:
# The code was removed by Watson Studio for sharing.

,PostCode,Latitude,Longitude
0,K2A,45.377763,-75.761418
1,K4A,45.467261,-75.466323
2,K1B,45.427258,-75.601085
3,K2B,45.361772,-75.788718
4,K4B,45.422538,-75.465064


In [15]:
coordinates_df.rename(columns = {"PostCode":"PostalCode"}, inplace = True)
coordinates_df.head()

,PostalCode,Latitude,Longitude
0,K2A,45.377763,-75.761418
1,K4A,45.467261,-75.466323
2,K1B,45.427258,-75.601085
3,K2B,45.361772,-75.788718
4,K4B,45.422538,-75.465064


In [16]:
ottawapopcoor_df = ottawapop_df.merge(coordinates_df, on="PostalCode", how="left")
ottawapopcoor_df.head()

,PostalCode,City,Neighbourhood,"Population, 2016","Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016",Latitude,Longitude
0,K2A,Ottawa,"Highland Park, McKellar Park,Westboro,Glabar P...",16790.0,7433.0,7205.0,45.377763,-75.761418
1,K4A,Ottawa,Fallingbrook,54524.0,18677.0,18544.0,45.467261,-75.466323
2,K1B,Ottawa,"Blackburn Hamlet, Pine View, Sheffield Glen",17110.0,6826.0,6611.0,45.427258,-75.601085
3,K2B,Ottawa,"Britannia,Whitehaven, Bayshore, Pinecrest",32260.0,15640.0,14719.0,45.361772,-75.788718
4,K4B,Ottawa,Navan,4793.0,1771.0,1732.0,45.422538,-75.465064


In [17]:
print('The dataframe has {} neighbourhoods.'.format(
        ottawapopcoor_df.shape[0])
    )

The dataframe has 40 neighbourhoods.


## Cluster the neighbourhoods in Ottawa

In [18]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    ------------------------------------------------------------
                       

In [19]:
address = 'Ottawa, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Ottawa are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Ottawa are 45.421106, -75.690308.


Create a map of Ottawa with neighbourhoods superimposed on top.

In [20]:
# create map of Ottawa using latitude and longitude values
map_ottawa = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighbourhood in zip(ottawapopcoor_df['Latitude'], ottawapopcoor_df['Longitude'], ottawapopcoor_df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ottawa)  
    
map_ottawa

Foursquare Credentials

In [21]:
# The code was removed by Watson Studio for sharing.

In [22]:
LIMIT = 100
radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=6000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
ottawa_venues = getNearbyVenues(names=ottawapopcoor_df['Neighbourhood'],
                                   latitudes=ottawapopcoor_df['Latitude'],
                                   longitudes=ottawapopcoor_df['Longitude']
                                  )


Highland Park, McKellar Park,Westboro,Glabar Park,Carlingwood
Fallingbrook
Blackburn Hamlet, Pine View, Sheffield Glen
Britannia,Whitehaven, Bayshore, Pinecrest
Navan
Orleans
Queensway, Copeland Park, Central Park, Bel Air,Carleton Heights
Cumberland
Queenswood
Eastern Nepean: Fisher Heights/ Parkwood Hills, Borden Farm,Pine Glen 
Riverview, Hawthorne, Canterbury, Hunt Club Park
Centrepointe, Meadowlands, City View, Craig Henry, Tangelwood, Grenfell Glen, Davidson Heights
Alta Vista, Billings Bridge
Bells Corners, Arlington Woods/Redwood, Qualicum, Crystal Beach
Beacon Hill, Cyrville, Carson Grove
Barrhaven
Overbrook, Forbes, Manor Park, Viscount Alexander Park, Finter Quarries
Beaverbrook, South March
Vanier, McKay Lake area
Katimavik-Hazeldean, Glen Cairn
Rockcliffe Park, New Edinburgh
Bridlewood
Manotick
Lower Town, Byward Market, Sandy Hill, University of Ottawa
Downtown
Centretown
Greely
Dalhousie Ward
Fallowfield Village, Cedarhill Estates, Orchard Estates
The Glebe, Old Ottawa S

In [24]:
ottawa_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Highland Park, McKellar Park,Westboro,Glabar P...",45.377763,-75.761418,Equator Coffee Westboro,45.391714,-75.753717,Coffee Shop
1,"Highland Park, McKellar Park,Westboro,Glabar P...",45.377763,-75.761418,Pure Kitchen,45.391683,-75.754797,Vegetarian / Vegan Restaurant
2,"Highland Park, McKellar Park,Westboro,Glabar P...",45.377763,-75.761418,S&G Fries,45.376841,-75.755878,Food Truck
3,"Highland Park, McKellar Park,Westboro,Glabar P...",45.377763,-75.761418,Bridgehead,45.389525,-75.759697,Coffee Shop
4,"Highland Park, McKellar Park,Westboro,Glabar P...",45.377763,-75.761418,Fratelli Entoca And Wine Bar,45.393409,-75.751298,Italian Restaurant


In [25]:
ottawa_gyms = ottawa_venues.loc[ottawa_venues['Venue Category'].isin(['Gym','Gym / Fitness Center'])]
ottawa_gyms.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
108,Fallingbrook,45.467261,-75.466323,Ray Friel Recreation Complex,45.471871,-75.492555,Gym
128,Fallingbrook,45.467261,-75.466323,Hostyle Conditioning,45.464877,-75.539758,Gym
140,Fallingbrook,45.467261,-75.466323,GoodLife Fitness,45.457172,-75.493859,Gym
150,Fallingbrook,45.467261,-75.466323,Ruddy Family YMCA-YWCA,45.481075,-75.509561,Gym
168,Fallingbrook,45.467261,-75.466323,Movati Athletic Orleans,45.449072,-75.514662,Gym / Fitness Center


In [26]:
ottawa_gyms.groupby(['Neighbourhood']).count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Alta Vista, Billings Bridge",2,2,2,2,2,2
Barrhaven,2,2,2,2,2,2
"Beacon Hill, Cyrville, Carson Grove",5,5,5,5,5,5
"Beaverbrook, South March",2,2,2,2,2,2
"Bells Corners, Arlington Woods/Redwood, Qualicum, Crystal Beach",1,1,1,1,1,1
"Blackburn Hamlet, Pine View, Sheffield Glen",3,3,3,3,3,3
Bridlewood,4,4,4,4,4,4
"Britannia,Whitehaven, Bayshore, Pinecrest",3,3,3,3,3,3
"Centrepointe, Meadowlands, City View, Craig Henry, Tangelwood, Grenfell Glen, Davidson Heights",3,3,3,3,3,3


In [27]:
# one hot encoding
ottawa_onehot = pd.get_dummies(ottawa_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ottawa_onehot['Neighbourhood'] = ottawa_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [ottawa_onehot.columns[-1]] + list(ottawa_onehot.columns[:-1])
ottawa_onehot = ottawa_onehot[fixed_columns]

ottawa_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Beer Bar,Beer Store,Big Box Store,Boat or Ferry,Bookstore,Botanical Garden,Breakfast Spot,Brewery,Buffet,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Café,Campground,Canal Lock,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Coffee Shop,College Cafeteria,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Drugstore,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hockey Arena,Home Service,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Lebanese Restaurant,Liquor Store,Malay Restaurant,Market,Mediterranean Restaurant,Memorial Site,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Motorcycle Shop,Movie Theater,Museum,Music Store,Neighborhood,New American Restaurant,Noodle House,Office,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Outlet Mall,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Pub,Record Shop,Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoothie Shop,Soccer Field,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Yoga Studio,Zoo
0,"Highland Park, McKellar Park,Westboro,Glabar P...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Highland Park, McKellar Park,Westboro,Glabar P...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,"Highland Park, McKellar Park,Westboro,Glabar P...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Highland Park, McKellar Park,Westboro,Glabar P...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [28]:
ottawa_onehot.shape

(3340, 175)